# Checks if Perfsonar data is indexed

This notebook checks number of indexed documents in all four doc types (latency, packetloss, throughput and traceroute) and alerts if any of them is 0. It sends mails to all the people substribed to that alert. It is run every 30 min from a cron job.

In [1]:
import subscribers, alerts
from datetime import datetime
from elasticsearch import Elasticsearch, exceptions as es_exceptions
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

In [2]:
ct  = datetime.utcnow()
ind = 'network_weather_2-%d.%d.%d' % (ct.year, ct.month, ct.day)
print(ind)
currentTime=int(round(datetime.now().timestamp()* 1000))
startTime=currentTime - 1800000
print('start time', startTime)
print('current time',datetime.now())

network_weather_2-2016.8.11
start time 1470882835614
current time 2016-08-10 22:03:55.614547


In [3]:
types_query={
   "size": 0,
   "query": {
        "bool":{
            "must": {
#                "term": {"_type":"latency"}
            },
            "filter":{
                "range":{"timestamp":{"gte": startTime,"lte": currentTime}}
            }
        }
   },
    "aggs" : {
        "docType" : {
            "terms" : { "field" : "_type" }
        }
    }
}

res = es.search(index=ind, body=types_query, request_timeout=120)
print(res)

{'took': 23, 'hits': {'total': 36554, 'hits': [], 'max_score': 0.0}, '_shards': {'total': 5, 'failed': 0, 'successful': 5}, 'aggregations': {'docType': {'sum_other_doc_count': 0, 'doc_count_error_upper_bound': 0, 'buckets': [{'key': 'latency', 'doc_count': 17082}, {'key': 'packet_loss_rate', 'doc_count': 17071}, {'key': 'traceroute', 'doc_count': 2202}, {'key': 'throughput', 'doc_count': 199}]}}, 'timed_out': False}


In [4]:
r=res['aggregations']['docType']['buckets']
problematic=[]
for t in r:
    print(t)
    if t['doc_count']==0:
        problematic.append(t['key'])
print('types with no docs:', problematic)

{'key': 'latency', 'doc_count': 17082}
{'key': 'packet_loss_rate', 'doc_count': 17071}
{'key': 'traceroute', 'doc_count': 2202}
{'key': 'throughput', 'doc_count': 199}
types with no docs: []


In [5]:
if len(problematic) > 0:
    S = subscribers.subscribers()
    A = alerts.alerts()

    testName = 'Perfsonar'
    subscribersToPerfsonar =  S.getSubscribers(testName)
    print (subscribersToPerfsonar)
    for subscriber in subscribersToPerfsonar:
        body = 'Dear ' + subscriber[0]+',\n\n'
        body = body + '\tthis mail is to let you that there is an issue in indexing Perfsonar data in UC Elasticsearch.\n'
        body = body + 'This is number of documents indexed in last 30 minutes per document type:\n'
        body = body + str(r)+'\n'
        body = body + '\nBest regards,\nATLAS AAS'
        body = body + '\n\n To change your alerts preferences please you the following link:\n'+subscriber[2]
        print(subscriber)
        A.sendMail(testName, subscriber[1], body)
        A.addAlert(testName, subscriber[0],str(r))

[['Ilija Vukotic', 'ivukotic@cern.ch', 'https://docs.google.com/forms/d/e/1FAIpQLSfwwtAvMrqp4Ot_LYfmNu75_v33dtAxiXg7ZvVdn1X5v7TEgg/viewform?edit2=2_ABaOnuf8m2xilpIjaTt3jlTf-1Fw9r42smGVmYATJ2Jap7Y_9Ym0hL_AzhCZIQ'], ['Ilija Vukotic', 'ilijav@gmail.com', 'https://docs.google.com/forms/d/e/1FAIpQLSfwwtAvMrqp4Ot_LYfmNu75_v33dtAxiXg7ZvVdn1X5v7TEgg/viewform?edit2=2_ABaOnuezuteui57-PrydNWrUuZf5fmChNqtjEeDab6h5V6lik_-x790uKsPu5Q']]
['Ilija Vukotic', 'ivukotic@cern.ch', 'https://docs.google.com/forms/d/e/1FAIpQLSfwwtAvMrqp4Ot_LYfmNu75_v33dtAxiXg7ZvVdn1X5v7TEgg/viewform?edit2=2_ABaOnuf8m2xilpIjaTt3jlTf-1Fw9r42smGVmYATJ2Jap7Y_9Ym0hL_AzhCZIQ']
Content-Type: text/plain; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit
Subject: Perfsonar
From: AAAS@mwt2.org
To: ivukotic@cern.ch

Dear Ilija Vukotic,

	this mail is to let you that there is an issue in indexing Perfsonar data in UC Elasticsearch.
This is number of documents indexed in last 30 minutes per document type:
[{'key': 'late